In [ ]:
import json
from datetime import date
from datetime import timedelta

import pandas as pd

pd.options.display.float_format = '{:,.3f}'.format
pd.options.display.max_rows = None
pd.options.display.width = None

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
%matplotlib inline

from city_stats import parse_repo
from nys_soda import get_data

CFG_FILE = 'config.json'

origin_date = date(year=1970, month=1, day=1)
def fix_plt_date(x, pos):
    actual_date = origin_date + mdates.num2timedelta(x)
    return actual_date.strftime('%Y-%m-%d')

with open(CFG_FILE) as f:
    cfg = json.load(f)

state_data = get_data(cfg['SOCRATA_TOKEN'])

In [ ]:
plot_columns = (['new_cases', 'new_cases_ave'], ['ratio', 'ratio_ave'])
plot_legends = (['New Cases', '7 day average'], ['Positive Test Ratio', '7 day average'])
rolling_period = 7

for (src, dst) in plot_columns:
    state_data[dst] = state_data[src].rolling(rolling_period).mean()

state_data.tail(15)


In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(15, 8))

for axis, column, legend in zip(axes, plot_columns, plot_legends):
    state_data[-30:][column].plot(ax=axis, rot=-60)
    axis.xaxis.set_major_locator(mdates.DayLocator())
    axis.xaxis.set_major_formatter(FuncFormatter(fix_plt_date))
    axis.tick_params(axis='x', which='minor', bottom=False)
    axis.grid(linestyle=':', linewidth='0.5')
    axis.legend(legend)
        

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(15, 8))

for axis, column, legend in zip(axes, plot_columns, plot_legends):
    state_data[column].plot(ax=axis, rot=-60)
    axis.xaxis.set_major_locator(mdates.WeekdayLocator())
    axis.xaxis.set_major_formatter(FuncFormatter(fix_plt_date))
    axis.tick_params(axis='x', which='minor', bottom=False)
    axis.grid(linestyle=':', linewidth='0.5')
    axis.legend(legend)

In [ ]:
state_data.index += timedelta(days=1)

city_data = parse_repo(cfg)
city_data.set_index('Date', inplace=True)

tabla = pd.concat([city_data['Delta_Cases'].tail(30), state_data['new_cases'].tail(30)], axis=1)
tabla = tabla.rename(columns={"Delta_Cases": "City Cases", "new_cases": "State Cases"})
tabla['Average'] = tabla.mean(axis=1)
tabla.tail(15)


In [ ]:

axis = tabla.plot(rot=-60, figsize=(15, 8), style=['-','-','r-'])
axis.xaxis.set_major_locator(mdates.DayLocator())
axis.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
axis.tick_params(axis='x', which='minor', bottom=False)
axis.grid(linestyle=':', linewidth='0.5')
